In [149]:
file1 = 'personal_info.csv'
file2 = 'vehicles.csv'
file3 = 'employment.csv'
file4 = 'update_status.csv'

Goal-1

In [150]:
import csv

In [151]:
files=[file1, file2, file3, file4]

In [152]:
from datetime import datetime
def dispatch(line):
    try:
        out = int(line)
    except ValueError:
        try:
            out = datetime.strptime(line, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            out = line
    return out

In [153]:
from collections import namedtuple
def create_nt_file1(file):
    with open(file) as f:
        reader = csv.reader(f, delimiter = ',' , quotechar = '"')
        personal_info = namedtuple('Personal_Info', [*next(reader)])
        for line in reader:
            #print(personal_info(*line))
            yield personal_info(*line)

In [155]:
def create_nt_file2(file):
    with open(file) as f:
        reader = csv.reader(f, delimiter = ',' , quotechar = '"')
        vehicles = namedtuple('Vehicles', [*next(reader)])
        for line in reader:
            #print(vehicles(*[dispatch(item) for item in line]))
            yield vehicles(*[dispatch(item) for item in line])

In [157]:
def create_nt_file3(file):
    with open(file) as f:
        reader = csv.reader(f, delimiter = ',' , quotechar = '"')
        employment = namedtuple('Employment', [*next(reader)])
        for line in reader:
            #print(employment(*[dispatch(item) for item in line]))
            yield employment(*[dispatch(item) for item in line])

In [159]:
def create_nt_file4(file):
    with open(file) as f:
        reader = csv.reader(f, delimiter = ',' , quotechar = '"')
        update_status = namedtuple('Update_Status', [*next(reader)])
        for line in reader:
            #print(update_status(*[dispatch(item) for item in line]))
            yield update_status(*[dispatch(item) for item in line])

In [154]:
create_nt_file1(file1)

<generator object create_nt_file1 at 0x73478bb49c80>

In [156]:
create_nt_file2(file2)

<generator object create_nt_file2 at 0x73478bb496d0>

In [158]:
create_nt_file3(file3)

<generator object create_nt_file3 at 0x73478bf5dcf0>

In [160]:
create_nt_file4(file4)

<generator object create_nt_file4 at 0x73478bad0200>

Goal 2

In [162]:
import itertools

In [167]:
def create_single_nt():
    for line in create_nt_file1(file1):
        fields1 = line._fields
        break
    for line in create_nt_file2(file2):
        fields2 = line._fields
        break
    for line in create_nt_file3(file3):
        fields3 = line._fields
        break
    for line in create_nt_file4(file4):
        fields4 = line._fields
        break
    fields = [*fields1, *fields2, *fields3, *fields4]
    new_fields = []
    for f in fields:
        if f not in new_fields:
            new_fields.append(f)
    combined = namedtuple('Combine', [*new_fields])
    return combined
 

In [168]:
new_nt = create_single_nt()

In [174]:
def combine():
    global combined
    combined= []
    for a,b,c,d in list(itertools.chain(zip(create_nt_file1(file1), create_nt_file2(file2), create_nt_file3(file3), create_nt_file4(file4)))):
        combined.append(new_nt(*a,*b[1:],*c[:3],*d[1:]))

In [175]:
combine()

Goal 3

In [217]:
def filter_stale_records():
    return filter(lambda x: x.last_updated >= datetime(2017,3,1,0,0,0),combined )

Goal 4

In [220]:
from collections import defaultdict
def create_dict():
    dict_male= defaultdict(int)
    dict_female = defaultdict(int)
    for gender, iter in itertools.groupby(sorted(filter_stale_records(), key = lambda x:x.gender), key = lambda x: x.gender):
        if gender == 'Female':
            for list in iter:
                dict_female[list.vehicle_make] += 1
        elif gender == 'Male':
            for list in iter:
                dict_male[list.vehicle_make] += 1
    #print(dict_female)
    #print(dict_male)
    print('female_dict {0}'.format({make:cnt for make, cnt in sorted(dict_female.items(),reverse = True, key = lambda x : x[1])}, end = ''))
    print('male_dict {0}'.format({make:cnt for make, cnt in sorted(dict_male.items(),reverse = True, key = lambda x : x[1])}, end = ''))

In [221]:
create_dict()

female_dict {'Chevrolet': 42, 'Ford': 42, 'GMC': 22, 'Mitsubishi': 22, 'Toyota': 20, 'Mercedes-Benz': 17, 'Dodge': 17, 'Lexus': 15, 'Pontiac': 14, 'Volvo': 13, 'Audi': 13, 'Mazda': 13, 'BMW': 12, 'Nissan': 12, 'Suzuki': 12, 'Buick': 11, 'Volkswagen': 10, 'Acura': 9, 'Kia': 9, 'Infiniti': 9, 'Land Rover': 8, 'Honda': 8, 'Oldsmobile': 8, 'Chrysler': 6, 'Cadillac': 6, 'Subaru': 6, 'Jeep': 5, 'Mercury': 5, 'Lotus': 5, 'Bentley': 4, 'Hyundai': 4, 'Lincoln': 4, 'Isuzu': 3, 'Plymouth': 3, 'Saturn': 3, 'Porsche': 3, 'Saab': 3, 'Jaguar': 3, 'Scion': 2, 'Aston Martin': 2, 'Lamborghini': 2, 'Bugatti': 1, 'Rolls-Royce': 1, 'Eagle': 1, 'Geo': 1, 'Morgan': 1, 'Austin': 1, 'Panoz': 1}
male_dict {'Ford': 40, 'Chevrolet': 30, 'GMC': 28, 'Mitsubishi': 28, 'Dodge': 22, 'Toyota': 21, 'Mercedes-Benz': 19, 'Volkswagen': 16, 'Audi': 14, 'Buick': 13, 'Mazda': 13, 'BMW': 12, 'Pontiac': 11, 'Mercury': 11, 'Volvo': 10, 'Cadillac': 9, 'Honda': 9, 'Subaru': 8, 'Hyundai': 8, 'Saab': 8, 'Acura': 7, 'Infiniti': 7, 'J